# Load packages

In [1]:
from core.DataLoader import *
from core.models.MLP import * 
from core.models.Classifier import *

In [2]:
clf_model_list ={
    "MLP": MLPClassifier,
}
clf_model_name_list = ['MLP']

# Load data

In [3]:
engine = 'postgres'
dl = DataLoader(engine)
one_file_dss, one_file_names = dl.get_one_file_ds(return_type='ds and names', datasets=['ssb', 'tpch', 'tpch_10', 'tpch_100', 'imdb'])

In [4]:
all_features = dl.all_features
classification_target = dl.classification_target
all_features = ['left_cardinality',
 'base_cardinality',
 'sel_of_pred_on_indexed_attr',
 'sel_of_pred_on_non_indexed_attr',
 'sel_of_join_pred',
 'left_ordered',
 'base_ordered',
 'left_indexed',
 'base_indexed',
 'result_size',
 'predicate_op_num_on_indexed_attr',
 'predicate_op_num_on_non_indexed_attr']

# Model accuracy with removed features

In [8]:
# Note: This cell is possibly going to take hours on your machine. 

present_idxs = list(range(len(one_file_dss)))
removed_acc = {}

for feat_idx, feat_in_group in enumerate(all_features):
    
    features = all_features.copy()
    
    features.remove(feat_in_group)
    
    print(f"remove feature {feat_in_group}: after remove length: %d"%len(features))
    
    model_accs = {
        'MLP': [],
        'RF(5)': [],
        'RF(U)': [],
        'DT(5)': [],
        'DT(U)': [],
        'LR': [],
        'SVM': [],
        'XGB': []
    }
    
    for i in present_idxs:  # range(len(one_file_dss)):
        print(f"Feature {feat_idx} / {len(all_features)}; Processing: {i} / {len(present_idxs)}")
        ds = one_file_dss[i]
        ds_name = one_file_names[i]

        # =========================
        X = ds[features]
        y = ds['optimal_decision']
        X_costs = ds[dl.regression_targets]

        from sklearn.model_selection import train_test_split
        from sklearn import preprocessing

        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)
        X_train_costs, _, _, _, = train_test_split(X_costs, y, train_size=0.8, random_state=1)
        X_train, X_test, y_train, y_test = \
            X_train.to_numpy(),X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

        X_train_weights = calculate_importance_from_costs(X_train_costs.to_numpy())
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        X_train_weights = preprocessing.MinMaxScaler().fit_transform(X_train_weights.reshape(-1,1)).flatten()
        # =========================

        accs = []
        for idx, model_name in enumerate(clf_model_name_list):
            model = clf_model_list[model_name.split('(')[0]]        
            if 'DT' in model_name or 'RF' in model_name:
                max_depth = model_name.split('(')[-1].split(')')[0]
                if max_depth.lower() == 'u':
                    max_depth = None
                else:
                    max_depth = int(max_depth)
                clf = model().fit(X_train, y_train, sample_weight=None, max_depth=max_depth)
            else:
                clf = model().fit(X_train, y_train, sample_weight=None, max_iter=200, weight_decay=0.00001)

            print(f"Accuray of {model}: {clf.score(X_test, y_test)}")
            model_accs[model_name].append(clf.score(X_test, y_test))
    
    removed_acc["%d" % feat_idx] = model_accs

  0%|          | 0/200 [00:00<?, ?it/s]

remove feature left_cardinality: after remove length: 11
Feature 0 / 12; Processing: 0 / 63
set features: 11


  0%|          | 0/200 [00:00<?, ?it/s]

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.99
Feature 0 / 12; Processing: 1 / 63
set features: 11


  0%|          | 0/200 [00:00<?, ?it/s]

...

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.995
Feature 11 / 12; Processing: 61 / 63
set features: 11


  0%|          | 0/200 [00:00<?, ?it/s]

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.985
Feature 11 / 12; Processing: 62 / 63
set features: 11


100%|██████████| 200/200 [00:20<00:00,  9.94it/s]

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.9975


# Count of binary joins that features are chosen as Top-2

In [12]:
top_feature_chosen = {}
for model_name in weighted_clf_model_name_list:
    model_features = []
    for i in range(len(present_idxs)):
        ablation_accs = []
        for feat_idx in range(len(all_features)):
            ablation_accs.append(removed_acc["%d" % feat_idx][model_name][i])
        ablation_accs = np.array(ablation_accs)
        if sum(ablation_accs) == ablation_accs[0] * len(ablation_accs):
            ablation_accs[range(len(ablation_accs))] = 0
        else:  
            sort_idx = np.argsort(ablation_accs)
            ablation_accs[sort_idx[0:2]] = 1
            ablation_accs[sort_idx[2:]] = 0
        model_features.append(ablation_accs)
    top_feature_chosen[model_name] = model_features

In [13]:
for model_name in weighted_clf_model_name_list:
    print(f"For model {model_name}")
    sum_feat = np.zeros(len(all_features))
    for idx, ds_name in zip(range(len(one_file_dss)), one_file_names):
#         print(f"{ds_name}: ", top_feature_chosen[model_name][idx])
        sum_feat += top_feature_chosen[model_name][idx]
    print(f"Summary for {model_name}: ", sum_feat)
    print()

For model MLP
ssb part/postgres_lineorder_part_optimal.csv:  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
ssb customer/postgres_lineorder_customer_optimal.csv:  [0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
ssb ddate/postgres_lineorder_ddate_optimal.csv:  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
ssb supplier/postgres_lineorder_supplier_optimal.csv:  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
ssb lineorder/postgres_part_lineorder_optimal.csv:  [0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
ssb lineorder/postgres_customer_lineorder_optimal.csv:  [1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
ssb lineorder/postgres_ddate_lineorder_optimal.csv:  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
ssb lineorder/postgres_supplier_lineorder_optimal.csv:  [1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
tpch customer/postgres_orders_customer_optimal.csv:  [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.]
tpch orders/postgres_customer_orders_optimal.csv:  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
tpch orders/postgres_lineitem_orders_optimal.csv:  [0. 0. 1. 1. 0. 0. 0. 0. 0. 